In [44]:
%load_ext autoreload
%autoreload 2
import hyperspy.api as hs
%matplotlib qt4
import matplotlib.pyplot as plt
plt.ioff()
import numpy as np
import cv2
from scipy.ndimage.interpolation import rotate
from skimage.transform import rescale
from hyperspy import project_routine

fname_diff="../../Data/AstartDataJulie/160512_SC239_X_overview_60k.blo"
fname_nwcontact = "../../Data/160309JulieContact/NWcontact_EDS Spectrum Image.dm3"

sd_data=np.load('../PROJECT/Project/Oct9/sd_picture.npy')
# se_data=np.load('../PROJECT/Project/Oct9/se_picture.npy')
# sd_data_new=np.load('../..//PROJECT/Project/Oct9/sd_picture_uint8_rotated.npy')
# se_data_sd=np.load('../..//PROJECT/Project/Oct9/se_picture_uint8.npy')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load (cropped) data

### Make image of appropriate edx data

In [ ]:
# se=hs.load('../..//Data/160506_SC239_X/C_EDS Spectrum Image.dm3')
# se.T.plot()
# se_sigcrop=se.isig[9.13:9.43]
# plt.figure()
# plt.imshow(se_sigcrop.data.mean(axis=2))
np.save('../..//Data/160506_SC239_X/C_EDS Spectrum Image.npy', se_sigcrop.data.mean(axis=2))

In [ ]:
sd.save("../../Data/sd_sigcrop.hdf5")
se.save("../../Data/se_sigcrop.hdf5")

In [82]:
pts1 = np.float32([[103,133],[89,72]]) #sd
#     pts2 = np.float32([[50,53],[131,53]]) #edx
pts2 = np.float32([[65,78],[118,68]]) #edx real one
angle1=np.arctan2(pts1[0,1]-pts1[1,1],pts1[0,0]-pts1[1,0])
angle2=np.arctan2(pts2[0,1]-pts2[1,1],pts2[0,0]-pts2[1,0])
rot = angle1-angle2

def scale_and_rotate(sd,sd_data,edx,se_data, rot,scalefactor=1):

    edx_scale = edx.axes_manager.as_dictionary()['axis-1']['scale']*1000
    sd_scale = sd.axes_manager.as_dictionary()['axis-1']['scale']
    scale = edx_scale/sd_scale*scalefactor

    se_data2 = rescale(se_data, scale)
    sd_data2 = rotate(sd_data, np.rad2deg(rot))
    sd_data2 -= np.min(sd_data2)
    sd_data2 = (sd_data2/np.max(sd_data2)*255).astype('uint8')
    se_data2 -= np.min(se_data2)
    se_data2 = (se_data2/np.max(se_data2)*255).astype('uint8')
    
    return (sd_data2, se_data2, scale)


def imreg(sd_data2, se_data2):
    im1 = se_data2
    im2 = sd_data2
#     print(im1.shape, im2.shape)
    w, h = im1.shape[::-1]

    methods = np.array(['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR','cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED'])
    m = 'cv2.TM_SQDIFF_NORMED'
    # not CCORR, CCOEFF, 
    # for m in methods:
    method=eval(m)
    res = cv2.matchTemplate(im2, im1, method)
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)

    cv2.rectangle(im2,top_left, bottom_right, 255, 1)

    plt.figure(m)
#     plt.subplot(121),plt.imshow(res,cmap = 'gray')
#     plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
#     plt.subplot(122),
    plt.imshow(im2,cmap = 'gray',interpolation='none',extent=[0,im2.shape[1],None,None])
#     plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
#     plt.suptitle(m)
#     plt.hold('off')
#     print(top_left,bottom_right)
    plt.show()
    plt.figure('se')
    plt.imshow(im1)
    
    return top_left, bottom_right,res

# def crop_sd(sd_data2, top_left, bottom_right):
#     sd_data_crop=sd_data2[top_left[1]:bottom_right[1],top_left[0]:bottom_right[0]]
#     return sd_data_crop

def crop_and_rotate_sd(sd, top_left, bottom_right, rot):
#     pts1 = np.float32([[103,133],[89,72]])
# #     pts2 = np.float32([[50,53],[131,53]])
#     pts2 = np.float32([[65,78],[118,68]]) #edx real one
#     angle1=np.arctan2(pts1[0,1]-pts1[1,1],pts1[0,0]-pts1[1,0])
#     angle2=np.arctan2(pts2[0,1]-pts2[1,1],pts2[0,0]-pts2[1,0])
#     rot = angle1-angle2
    print('rotate')
    sd2 = sd.rotate(angle=np.rad2deg(rot), reshape=True, rotate_dimension='navigation')
    
#     sd2.crop('x',top_left[0], bottom_right[0])
#     sd2.crop('y',top_left[1], bottom_right[1])
    print('crop')
    sd2=sd2.inav[top_left[0]:bottom_right[0],top_left[1]:bottom_right[1]]
    
    return sd2

# def connect(sd2, edx, scale, tl):
#     offset = tl
#     sd2.plot(colorbar=False)
#     edx.plot()
#     sd2.connect(edx, scale, offset)
    
def rebin_se(se, scale):
    print('rebin')
    return se.linear_bin([scale, scale, 1], crop='False')
    
def calculate(se_data,sd_data,scalenum=False, rotnum=False,factnum=False):
    pts1 = np.float32([[103,133],[89,72]]) #sd
    pts2 = np.float32([[65,78],[118,68]]) #edx real one
    angle1=np.arctan2(pts1[0,1]-pts1[1,1],pts1[0,0]-pts1[1,0])
    angle2=np.arctan2(pts2[0,1]-pts2[1,1],pts2[0,0]-pts2[1,0])
    rot = angle1-angle2
    # plt.figure('se f')
    # plt.imshow(se.data.mean(axis=2))

    sd_data2, se_data2, sd_scale = scale_and_rotate(sd,sd_data,se,se_data, rot)
    tlold,brold,res = imreg(sd_data2, se_data2)
    res=np.reshape(res,-1)
    locs=np.argsort(res)
    res = res[locs[0]]
    print('res',res,'\n')
    plt.ioff()

    oldrot = rot
    oldres = res
    newrot=rot
    newres=res
    oldfact=1
    newfact=1
    oldscale=1
    newscale=1
    # se_data[se_data<.61]=.1
    if not scalenum:
        scale0=oldscale
        scale1=oldscale
        scalenum=1
    else:
        scale0 = .9
        scale1 = 1.
    if not rotnum:
        rot0=0
        rot1=0
        rotnum =1
        rot=oldrot
    else:
        rot0 = .03
        rot1 = .03
        rot=oldrot
        
    for scale in np.linspace(scale0,scale1,scalenum):
        for fact in np.linspace(.8,1,factnum):
            print(rot*(1-rot0),rot*(1+rot1),rotnum)
            for r in np.linspace(rot*(1-rot0),rot*(1+rot1),rotnum):
                sd_data2, se_data2, sd_scale = scale_and_rotate(sd,sd_data,se,se_data, r,scale)
                tl,br,res = imreg((sd_data2*fact).astype('uint8'), se_data2)
                res=np.reshape(res,-1)
                locs=np.argsort(res)
                res = res[locs[0]]
                if res < newres:
                    newres = res
                    newrot=r
                    tlnew=tl
                    brnew=br
                    newfact=fact
                    newscale=scale
                    
    return newfact, newscale, newrot, newres

def plot_all(se_data,sd_data, se, rot, scale, fact, fullres, se2=False,R=False,S=False):
    sd_data2, se_data2, sd_scale = scale_and_rotate(sd,sd_data,se,se_data, rot,scale)
    tl,br,res = imreg((sd_data2*fact).astype('uint8'), se_data2)
    if not se2:
        se2 = rebin_se(se, 1/sd_scale)
    # import cv2
    se_data3=se2.data.mean(axis=2)
    se_data3=(se_data3/(se_data3.max()-se_data3.min())*255).astype('uint8')

    sd_data3, se_data2fwafwa, sd_scdadadale = scale_and_rotate(sd,sd.data.mean(axis=(2,3)),se,se_data, rot,scale)
    sd_data3=(sd_data3/(sd_data3.max()-sd_data3.min())*255)#.astype('uint8')
    sd_data3 -=50
    sd_data3[sd_data3<0]=0
    sd_data3=sd_data3.astype('uint8')
    ime=cv2.cvtColor(se_data3,cv2.COLOR_GRAY2RGB)
    ims=cv2.cvtColor(sd_data3,cv2.COLOR_GRAY2RGB)
    ime[:,:,1:]=0
    s_img=ims.copy()
    l_img=ime

    by=tl[1]
    bx=tl[0]
    height=br[0]-tl[0]
    width=br[1]-tl[1]

    ims[tl[1]:tl[1]+ime.shape[0],tl[0]:tl[0]+ime.shape[1]] = ime
    ims[:,90:100] = s_img[:,90:100]
    ims[:,120:130] = s_img[:,120:130]
    
    plt.figure('im')
    sd_scale = sd.axes_manager.as_dictionary()['axis-1']['scale']
    plt.imshow(ims,interpolation='none',extent=1/np.cos(rot+np.pi/2)*sd_scale*np.array([0,sd.data.shape[0],0,sd.data.shape[1]]))
    plt.xlabel('nm')
    plt.ylabel('nm')
    plt.show()
    
#     print(rot,scale,fullres)

    plt.savefig('../PROJECT/thesis/fig/other/edxinsped/allnew/'+R+S+str(np.rad2deg(rot))[:8]+'_'+str(scale)[:8]+'_'+str(fullres)[:5]+'.png')

In [83]:
sd=hs.load("../Data/sd_sigcrop.hdf5")
# se=hs.load('../Data/160506_SC239_X/C_EDS Spectrum Image.dm3')
sd_data=np.load('../PROJECT/Project/Oct9/sd_picture.npy')
se_data = np.load('../Data/160506_SC239_X/C_EDS Spectrum Image.npy')

fact = 5
for scale in [0,5]:
    for rot in [0,5]:
        fact2,scale2,rot2, res = calculate(se_data,sd_data,scale,rot,fact)
        if rot == 0:
            R=''
        else:
            R='R'
        if scale==0:
            S=''
        else:
            S='S'
        if rot == 0 and fact == 0:
            se2 = plot_all(se_data,sd_data,se, rot2,scale2,fact2,fullres=res,R=R,S=S)
        else:
            plot_all(se_data,sd_data, se,rot2,scale2,fact2,se2=se2,fullres=res,R=R,S=S)

res 0.133206 

-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
rebin
res 0.133206 

-1.56161340475 -1.65820804834 5
-1.56161340475 -1.65820804834 5
-1.56161340475 -1.65820804834 5
-1.56161340475 -1.65820804834 5
-1.56161340475 -1.65820804834 5
rebin
res 0.133206 

-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
-1.60991072655 -1.60991072655 1
-1.60991072655 

In [8]:
print(0.926262626263,newscale)
print(newrot)

0.926262626263 1
-1.61681034395


In [6]:
# sd_data2, se_data2, sd_scale = scale_and_rotate(sd,sd_data,se,se_data, newrot,newscale)
# se2 = rebin_se(se, 1/sd_scale)
print(newscale)
# print(fac_orig)
scale_orig=1
fact_orig=1
rot_orig=np.deg2rad(-92.2411)
scale_new=newscale
fact_new=newfact
rot_new=newrot

# root=rot_orig
root=newrot
scaale=scale_orig
# scaale=newscale

sd_data2, se_data2, sd_scale = scale_and_rotate(sd,sd_data,se,se_data, root,scaale)
tl,br,res = imreg((sd_data2*fact_new).astype('uint8'), se_data2)
# se2 = rebin_se(se, 1/sd_scale)
# import cv2
se_data3=se2.data.mean(axis=2)
se_data3=(se_data3/(se_data3.max()-se_data3.min())*255).astype('uint8')


sd_data3, se_data2fwafwa, sd_scdadadale = scale_and_rotate(sd,sd.data.mean(axis=(2,3)),se,se_data, root,scaale)
# sd_data3, se_data2fwafwa, sd_scdadadale = scale_and_rotate(sd,sd_data2,se,se_data, rot_orig,1)

sd_data3=(sd_data3/(sd_data3.max()-sd_data3.min())*255)#.astype('uint8')
sd_data3 -=50
sd_data3[sd_data3<0]=0
sd_data3=sd_data3.astype('uint8')
ime=cv2.cvtColor(se_data3,cv2.COLOR_GRAY2RGB)
ims=cv2.cvtColor(sd_data3,cv2.COLOR_GRAY2RGB)
# ims[:,:,1:]=0
ime[:,:,1:]=0
s_img=ims.copy()
l_img=ime

by=tl[1]
bx=tl[0]
height=br[0]-tl[0]
width=br[1]-tl[1]

ims[tl[1]:tl[1]+ime.shape[0],tl[0]:tl[0]+ime.shape[1]] = ime
ims[:,90:100] = s_img[:,90:100]
ims[:,120:130] = s_img[:,120:130]
# plt.close('ims2')
plt.figure('ims3')
sd_scale = sd.axes_manager.as_dictionary()['axis-1']['scale']
plt.imshow(ims,interpolation='none',extent=1/np.cos(root+np.pi/2)*sd_scale*np.array([0,sd.data.shape[0],0,sd.data.shape[1]]))
plt.xlabel('nm')
plt.ylabel('nm')
plt.show()
plt.savefig('../PROJECT/thesis/fig/other/edxinsped/rot5')
# sd2.save('../Data/sd-mean.hdf5')

1


In [9]:
# plt.show()
print(rot_orig,180/np.pi*newrot)

-1.60991090066 -92.6364089812


#### Do this for 'other data'

In [ ]:
sd=hs.load("../../Data/sd_sigcrop.hdf5")
# se=hs.load('../..//Data/160506_SC239_X/C_EDS Spectrum Image.dm3')
se_data = np.load('../../Data/160506_SC239_X/C_EDS Spectrum Image.npy')
sd_data2, se_data2, sd_scale = scale_and_rotate(sd,sd_data,se,se_data, rot)
tl,br,res = imreg(sd_data2, se_data2)

sef=hs.load('../../Data/160506_SC239_X/I_EDS Spectrum Image.dm3')
scalef=scalelist[8]
scaleref=scalelist[2]
# scalef/=scaleref
scalenew=(sd_scale*scalef/scaleref)
minus2 = np.asarray([tl[1],tl[0]])
tlf=tuple(np.ceil(((np.asarray(tllist[8])-minus2))*scalenew).astype('int'))
brf=tuple(np.ceil(((np.asarray(brlist[8])-minus2))*scalenew).astype('int'))

sd2 = crop_and_rotate_sd(sd, tlf, brf, rot)
# scalerebin=1/(scalef*sd_scale/scaleref)
se2 = rebin_se(sef, scalerebin)
# se2.save('I_se.hdf5')
# sd2.save('I_sd.hdf5')
print(sd2.data.shape,se2.data.shape)

In [ ]:
print(sd2.data.shape,se2.data.shape)

In [ ]:
print (np.argsort(np.array(res).reshape(-1)))
locs=np.argsort(np.array(res).reshape(-1))
print(np.array(res).reshape(-1)[locs[:10]])

least100 = np.array(res).reshape(-1)[locs[:100]]
least0 = np.array(res).reshape(-1)[locs[0]]

                                  
plt.figure()
plt.scatter(np.arange(100),100*(1-(least100-least0)/least0))
plt.ylabel('Percent')
plt.xlabel('Pixel')

In [ ]:
print(np.array(res).reshape(-1)[locs[-1]])

In [ ]:
se=hs.load('../../Data/160506_SC239_X/C_EDS Spectrum Image.dm3')
se_data = se.mean(axis=2).data
sd_data2, se_data2, sd_scale = scale_and_rotate(sd, sd_data, se, se_data)
tl, br = imreg(sd_data2, se_data2)

In [ ]:
tl,br = imreg(sd_data, se_data)

# Find angle and scalefactor

# Scale and rotate images

# ImReg

# Rotate SD

## Connect EDX and SD

In [ ]:
plt.close('all')
offset = (67,76)
sd2.plot(colorbar=False)
edx.plot()

In [ ]:
sd2.connect(edx, scale, offset)

# The other data....

## Use survey images instead

In [7]:
%load_ext autoreload
%autoreload 2
%matplotlib qt4
import matplotlib.pyplot as plt
plt.close('all')
import numpy as np
import cv2
from hyperspy import project_routine
import hyperspy.api as hs
from hyperspy.io_plugins.digital_micrograph import file_reader
PATH='../Data/160506_SC239_X/'
END='_SI Survey Image.dm3'

l = ['A','B','C','D','E','F','G','H','I','J']
data=[]
rects=[]
ref='../Data/160506_SC239_X/HAADF300k.dm3'
for i,a in enumerate(l[:]):
#     if a in ['A','B','G','H']:
    if a not in ['D','E']:
        continue
    ok=False
    for g in ['TagGroup0','TagGroup1','TagGroup2','TagGroup3']:
        try:
            if file_reader(PATH+a+END)[0]['original_metadata']['DocumentObjectList'][
'TagGroup0']['AnnotationGroupList'][g]['Name']=='Spectrum Image':
                rect = file_reader(PATH+a+END)[0]['original_metadata']['DocumentObjectList']['TagGroup0']['AnnotationGroupList'][g]['Rectangle']
                print (i,'ok')
                ok=True
        except:
            pass
    if not ok:
        print(file_reader(PATH+a+END)[0]['original_metadata']['DocumentObjectList'][
'TagGroup0']['AnnotationGroupList'])
    data.append(PATH+a+END)
    rects.append(rect)
tl, br, scale, im, allres  =project_routine.prepare_data(data,ref, rects)

mini=0
# for val in np.arange(40,70,1):
#     mini=0
#     print(val)
#     tl, br, scale, im,allres  =project_routine.prepare_data(data,ref, rects,val)
#     for a in allres:
#         mini += np.min(a)
#     print(mini)


tllist, brlist, scalelist, im,allres  =project_routine.prepare_data(data,ref, rects,68)

for t,b in zip(tllist,brlist):
#     t=tllist[2]
#     b=brlist[2]
    plt.figure()
    imC = cv2.rectangle((t[0],t[1]),[b[0],b[1]])
#     imC = im[t[1]:b[1],t[0]:b[0]]
    plt.imshow(imC)

# plt.figure()
# im=cv2.cvtColor(im,cv2.COLOR_GRAY2RGB)
# for t,b in zip(tllist,brlist):
#     cv2.rectangle(im, t,b,(255,0,0),2)
# plt.imshow(im,interpolation='none')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
3 ok
4 ok


TypeError: Required argument 'pt1' (pos 2) not found

In [41]:
ref='../Data/160506_SC239_X/HAADF300k.dm3'
r=hs.load(ref)
s=r.axes_manager.as_dictionary()['axis-1']['scale']
print(s)
plt.close('all')
print(tllist)
t=tllist[1]
b=brlist[1]
plt.figure()
imC = cv2.rectangle(im,t,b,0,1)
imC = imC[150:150+220,580:800]
#     imC = im[t[1]:b[1],t[0]:b[0]]
plt.imshow(imC)#,extent=[0,s*imC.shape[0],0,s*imC.shape[1]])
plt.xlabel('nm')
plt.ylabel('nm')

0.613493204117
[(439, 283), (687, 247)]


In [43]:
print(rotnew)

NameError: name 'rotnew' is not defined

In [ ]:
# sd=hs.load('../../Data/AstartDataJulie/160512_SC239_X_overview_60k.blo')
# sdmean=sd.mean(axis=(2,3))
# sdrot,se,sc=scale_and_rotate(sd, sdmean, se, hs.load(ref),rot)
sdmean.data -=sdmean.data.min()
sdrot = rotate((sdmean.data/sdmean.data.max()*255).astype('uint8'), np.rad2deg(rot))
plt.close('all')
plt.figure('sdrot')
scale_sd=sd.axes_manager.as_dictionary()['axis-1']['scale']
print(scale)
cv2.rectangle(sdrot,tl,br,0,1)
plt.imshow(sdrot,extent=[0,sdrot.shape[1]*scale_sd,0,sdrot.shape[0]*scale_sd],interpolation='none')
plt.xlabel('nm')
plt.ylabel('nm')
t=tllist[2]
b=brlist[2]
plt.figure('only c')
imC = im[t[1]:b[1],t[0]:b[0]]
scale=scalelist[2]
imC = cv2.cvtColor(imC,cv2.COLOR_GRAY2RGB)
# imC[:,:,1:]=0
plt.imshow(imC,extent=[0,imC.shape[1]*scale,0,imC.shape[0]*scale],interpolation='none',alpha=1)
plt.xlabel('nm')
plt.ylabel('nm')

In [ ]:
# s_img = cv2.imread("smaller_image.png", -1)
for c in range(0,3):
    l_img[y_offset:y_offset+s_img.shape[0], x_offset:x_offset+s_img.shape[1], c] =  
    s_img[:,:,c] * (s_img[:,:,3]/255.0) +  l_img[y_offset:y_offset+s_img.shape[0], x_offset:x_offset+s_img.shape[1], c] * (1.0 - s_img[:,:,3]/255.0)

In [ ]:
plt.interactive(0)
plt.ioff()
x=['A','B','C','D','E','F','G','H','I','J']
plt.close('all')
allres=np.array(allres)
plt.figure()
plt.plot([0,1],100*(1-allres[:2])/(1-allres[2]),'bs',markersize=10)
# plt.xticks(np.arange(len(allres[:2])),x[:2])
plt.hold(True)
plt.plot(np.arange(3,len(allres)),100*(1-allres[3:])/(1-allres[2]),'ro',markersize=10)
plt.plot([2],100,'r*',markersize=20)
plt.xticks(np.arange(len(allres)),x)
plt.xlim([-1,10])
plt.ylim([None, None])
plt.xlabel('Dataset')
plt.ylabel('Percent')
plt.show()


In [ ]:
plt.close('all')

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib qt4
import matplotlib.pyplot as plt
plt.close('all')
import numpy as np
from hyperspy import project_routine
import hyperspy.api as hs
from hyperspy.io_plugins.digital_micrograph import file_reader
PATH='../..//Data/unheated/'
END='_SI Survey Image.dm3'

l = ['A','B','C','D','E','F','G','H','I']
data=[]
rects=[]
ref='../..//Data/unheated/HAADF_300k.dm3'
for i,a in enumerate(l[:]):
#     if a in ['B','D']:
#     if a not in ['A']:
#         continue
    ok=False
    for g in ['TagGroup0','TagGroup1','TagGroup2','TagGroup3']:
        try:
            if file_reader(PATH+a+END)[0]['original_metadata']['DocumentObjectList'][
'TagGroup0']['AnnotationGroupList'][g]['Name']=='Spectrum Image':
                rect = file_reader(PATH+a+END)[0]['original_metadata']['DocumentObjectList']['TagGroup0']['AnnotationGroupList'][g]['Rectangle']
                print (i,'ok')
                ok=True
        except:
            pass
    if not ok:
        print(file_reader(PATH+a+END)[0]['original_metadata']['DocumentObjectList'][
'TagGroup0']['AnnotationGroupList'])
    data.append(PATH+a+END)
    rects.append(rect)
# mini=0
# for val in np.arange(80,90,1):
#     mini=0
#     print(val)
#     tl, br, scale, im,allres  =project_routine.prepare_data(data,ref, rects,val)
#     for a in allres:
#         mini += np.min(a)
#     print(mini)

tl, br, scale, im,allres  =project_routine.prepare_data(data,ref, rects,81)


# plt.close('all')
plt.figure()
im=cv2.cvtColor(im,cv2.COLOR_GRAY2RGB)
for t,b in zip(tl,br):
    cv2.rectangle(im, t,b,(255,0,0),2)

plt.imshow(im,interpolation='none')

In [ ]:
x=['A','B','C','D','E','F','G','H','I']
# plt.close('all')
allres=np.array(allres)
plt.figure()
a = np.array([allres[1],allres[3]])
plt.plot([1,3],100*(1-a)/(1-allres[0]),'bs',markersize=10)
# plt.xticks(np.arange(len(allres[:2])),x[:2])
plt.hold(True)
a = np.array([allres[2],allres[4],allres[5],allres[6],allres[7],allres[8]])
plt.plot([2,4,5,6,7,8],100*(1-a)/(1-allres[0]),'ro',markersize=10)
plt.plot([0],100,'r*',markersize=20)
plt.xticks(np.arange(len(allres)),x)
plt.xlim([-1,9])
plt.xlabel('Dataset')
plt.ylabel('Percent')

In [ ]:
# plt.figure()
# im=cv2.cvtColor(im,cv2.COLOR_GRAY2RGB)
# for t,b in zip(tl,br):
#     cv2.rectangle(im, t,b,(255,0,0),2)

plt.imshow(im,interpolation='none')

In [ ]:
print(tl[1])
print(br[1])
print(scale[1])
# se=hs.load('../..//Data/160506_SC239_X/D_EDS Spectrum Image.dm3')
# connect(sd2, se, sd_scale*scale[1], tl[1])

In [ ]:
print(sd_scale, scale[1])

In [ ]:
%matplotlib qt4
import hyperspy.api as hs
s=hs.load('../../Data/unheated/D_EDS Spectrum Image.dm3')
s=s.sum('x').sum('y')
fig=plt.figure()
s.metadata
s.plot()
# s.add_elements(['Au'])
# s.add_lines()
# s.plot(background_windows=[[1,2,5,6]])

In [ ]:
import matplotlib.pyplot as plt
plt.close('all')

In [ ]:
import hyperspy as h
h.io_plugins.digital_micrograph.file_reader

# Old functions

In [ ]:
edx.plot()
sd2.connect(edx, scale,offset)
sd2.axes_manager.navigation_axes

In [ ]:
fname_diff="../../Data/AstartDataJulie/160512_SC239_X_overview_60k.blo"
fname_crop = "../../Data/AstartDataJulie/diff-contact.blo"
fname_smallgrains = "../../Data/160309JulieContact/SmallGrains_EDS Spectrum Image.dm3"
fname_nwcorner = "../../Data/160309JulieContact/NWcorner_EDS Spectrum Image.dm3"
fname_nwcontact = "../../Data/160309JulieContact/NWcontact_EDS Spectrum Image.dm3"
fname_contact='../../Data/160309JulieContact/contact_EDS Spectrum Image.dm3'

#USE HAADF, NOT BFTEM
fname_overview='../../Data/160309JulieContact/BFTEM_overview.dm3'
fname_haadf1=   '../../Data/160309JulieContact/HAADF_250k_3C.dm3'
fname_haadf2=   '../../Data/160309JulieContact/HAADF_250k_8C.dm3'
fname_haadf3=   '../../Data/160309JulieContact/HAADF_500k_8C.dm3'

In [ ]:
import hyperspy.api as hs
e0=hs.load(fname_haadf1)
e1=hs.load(fname_nwcontact)
e2=hs.load(fname_nwcorner)
e3 =hs.load(fname_smallgrains)
e4 = hs.load(fname_contact)
e1d=e1.data.mean(axis=2)
e2d=e2.data.mean(axis=2)
e3d = e3.data.mean(axis=2)
e4d = e4.data.mean(axis=2)

In [ ]:
e0s = e0.axes_manager.as_dictionary()['axis-1']['scale']
e1s = e1.axes_manager.as_dictionary()['axis-1']['scale']*1000
e2s = e2.axes_manager.as_dictionary()['axis-1']['scale']
e3s = e3.axes_manager.as_dictionary()['axis-1']['scale']
e4s = e4.axes_manager.as_dictionary()['axis-1']['scale']*1000

In [ ]:
e1dr = rescale(e1d, e1s/e0s)
e2dr = rescale(e2d, e2s/e0s)
e3dr = rescale(e3d, e3s/e0s)
e4dr = rescale(e4d, e4s/e0s)
# plt.imshow(e1dr)
# plt.show()

In [ ]:
e1dr -= e1dr.min()
e1dr = (e1dr/e1dr.max()*255).astype('uint8')
e2dr -= e2dr.min()
e2dr = (e2dr/e2dr.max()*255).astype('uint8')
e3dr -= e3dr.min()
e3dr = (e3dr/e3dr.max()*255).astype('uint8')
e4dr -= e4dr.min()
e4dr = (e4dr/e4dr.max()*255).astype('uint8')
e0d=e0.data
e0d -=e0d.min()
e0d = (e0d/e0d.max()*255).astype('uint8')
# plt.figure()
# plt.imshow(e2dr)
# plt.show()

In [ ]:
e0 = hs.load(fname_haadf1)
e0d=e0.data
e0d -=e0d.min()
e0d = (e0d/e0d.max()*255).astype('uint8')
im2 = e0d
for e in [e1dr, e2dr, e3dr, e4dr]:
    project_routine.imreg(e, im2, show=True, name='1')
#     newpos = project_routine.imreg(e4dr, im2)

offset = (67,76)

In [ ]:
# plt.close('all')
e4.plot()

top_left, bottom_right = newpos
tlref,brref            = refpos
scale=e0s/sd_scale

tl1 = int(offset[0]+(top_left[0]-tlref[0])*scale)
tl2 = int(offset[1]+(top_left[1]-tlref[1])*scale)
tl = (tl1,tl2)

br = (tl[0]+int((bottom_right[0]-top_left[0])*scale),tl[1]+int((bottom_right[1]-top_left[1])*scale))

# cv2.rectangle(sd_data2,tl, br, 255, 1)

plt.figure(); plt.imshow(sd_data2)

In [ ]:
e4_offset = project_routine.find_offset(e0d, e1dr, e4dr, e0s/sd_scale, (67,76))
print(e4_offset)

In [ ]:
sd2.plot()
e4.plot()

In [ ]:
sd2.connect(e4, e4s/sd_scale, e4_offset[0])

In [ ]:
se = hs.load("../../Data/160309JulieContact/NWcontact_EDS Spectrum Image.dm3")